In [1]:
import time
import types
import random
import textwrap
import quicktoken
import requests
from itertools import islice
from bs4 import BeautifulSoup

### Prepare ngrams and Build Markov Chain

In [2]:
def build_ngrams(itr, n=2):
    result = []
    index = 0
    while index + n-1 < len(itr):
        tempVar = []
        tempNum = 0
        while tempNum < n:
            tempVar.append(itr[index + tempNum ])
            tempNum = tempNum + 1
        tempVar = tuple(tempVar)
        result.append(tempVar)
        index = index + 1
    result = (i for i in result)
    return result

a = build_ngrams(range(10), n=2)
assert hasattr(a, '__iter__') and not isinstance(a, list)
al = list(a)
assert al == [(i,i+1) for i in range(9)]

b = build_ngrams(range(10), n=5)
assert hasattr(b, '__iter__') and not isinstance(b, list)
bl = list(b)
assert bl == [(i,i+1,i+2,i+3,i+4) for i in range(6)]

assert list(build_ngrams('one two three four five six seven'.split(' '), n=5)) == \
    [('one','two','three','four','five'),
     ('two','three','four','five','six'),
     ('three','four','five','six','seven')]

In [5]:
def build_chain(ngrams, chain=None):

    if chain != None:
        chain = chain
    else:
        chain = {}
        
    ngrams = list(ngrams)
    n= len(ngrams[0])
    
    for gram in ngrams:
        key = []
        for i in range(n-1):
            key.append(gram[i])
        key = tuple(key)
        data = gram[n-1]
        if key in chain:
            chain[key].append(data)
        else:
            chain[key] = [data]
   
    return chain
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)
assert chain[(0,0)]==[7, 10, 0, 3, 4]
assert chain[(4,2)]==[1, 3, 8, 3, 7, 1, 10, 2, 8]
assert len(chain.keys())==111

In [9]:
def generate_sequence(chain, m):
    a = random.choice(list(chain.keys()))
    
    length = m
    result = []
    tupLen = len(a)
    index = 0
    
    for num in a:
        result.append(num)
    length = length - tupLen
    while length > 0:
        
        tempKey = []
        
        for i in range(tupLen):
            tempKey.append(result[index + i])
            
        tempKey = tuple(tempKey)
        b = random.choice(chain[tempKey])
        result.append(b)
        index = index + 1
        length = length -1
    return result
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
chain(build_ngrams(seq1, n=5))

## Build and Scrape Lyrics from a internet site

In [11]:
def get_lyric_urls(url):
    index = requests.get(url)
    soup = BeautifulSoup(index.text, 'html.parser')
    
    lyric_paths = [link.get('href') for link in
                   soup.find_all('a')]
    urls= []
    
    for index in range(len(lyric_paths)):
        if lyric_paths[index] == None :
            continue
        elif "news" in lyric_paths[index]:
            continue
        elif "-lyrics" in lyric_paths[index]:
            urls.append(lyric_paths[index])
        
    print(urls)
    lyric_urls = [i.strip() for i in urls]
    return lyric_urls

lyric_urls = get_lyric_urls(str(input('Enter MetroLyrics URL here:')))

Enter MetroLyrics URL here:http://www.metrolyrics.com/metallica-lyrics.html
['http://www.metrolyrics.com/metallica-lyrics.html', 'http://www.metrolyrics.com/nothing-else-matters-lyrics-metallica.html', 'http://www.metrolyrics.com/enter-sandman-lyrics-metallica.html', 'http://www.metrolyrics.com/unforgiven-lyrics-metallica.html', 'http://www.metrolyrics.com/fuel-lyrics-metallica.html', 'http://www.metrolyrics.com/metallica-lyrics.html', 'http://www.metrolyrics.com/nothing-else-matters-lyrics-metallica.html', 'http://www.metrolyrics.com/enter-sandman-lyrics-metallica.html', 'http://www.metrolyrics.com/unforgiven-lyrics-metallica.html', 'http://www.metrolyrics.com/fuel-lyrics-metallica.html', 'http://www.metrolyrics.com/fade-to-black-lyrics-metallica.html', 'http://www.metrolyrics.com/one-lyrics-metallica.html', 'http://www.metrolyrics.com/master-of-puppets-lyrics-metallica.html', 'http://www.metrolyrics.com/whiskey-in-the-jar-lyrics-metallica.html', 'http://www.metrolyrics.com/the-unforg

In [12]:
def get_lyric(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    html_lyrics = soup.find_all("p", class_= "verse")
    html_lyrics = [l.getText() for l in html_lyrics]
    print(html_lyrics)
    return '\n'.join(html_lyrics)

In [14]:
def get_all_lyrics(lyric_urls):
    for url in lyric_urls:
        time.sleep(1.0)
        yield get_lyric(url)
        
lyrics = get_all_lyrics(lyric_urls)

In [15]:
with open('data/all_artist_lyrics.txt', 'w') as f:
    for lyric in lyrics:
        f.write(lyric.replace('\r\n', '\n'))
        f.write('\n')
        
assert True

[]
["So close, no matter how far\nCouldn't be much more from the heart\nForever trust in who we are\nAnd nothing else matters", "Never opened myself this way\nLife is ours, we live it our way\nAll these words, I don't just say\nAnd nothing else matters", 'Trust I seek and I find in you\nEvery day for us something new\nOpen mind for a different view\nAnd nothing else matters', 'Never cared for what they do\nNever cared for what they know\nAnd I know', "So close, no matter how far\nCouldn't be much more from the heart\nForever trusting who we are\nAnd nothing else matters", 'Never cared for what they do\nNever cared for what they know\nAnd I know, yeah', 'I never opened myself this way', "Life is ours, we live it our way\nAll these words, I don't just say\nAnd nothing else matters\nTrust I seek and I find in you", "Every day for us something new\nOpen mind for a different view\nAnd nothing else matters\nNever cared for what they say\nNever cared for games they play\nNever cared for what 

['Gimme fuel\nGimme fire\nGimme that which I desire\nOoh\nYeah\nYeah', 'Turn on, I see red\nAdrenaline crash, crack my head\nNitro junkie, paint me dead\nAnd I see red', 'A hundred plus through black and white', 'War horse', "War head\nFuck 'em, man", 'White knuckle tight', 'Through black and white\nOoh', 'On I burn\nFuel is pumping engines\nBurning hard\nLoose and clean\nOoh\nAnd then I burn\nTurning my direction\nQuench my thirst with gasoline\nSo gimme fuel\nGimme fire\nGimme that which I desire\nYeah\nTurn on beyond the bone\nSwallow future, spit out hope\nBurn your face upon the chrome\nYeah, oh yeah\nTake the corner, going to crash\nHeadlights\nHeadlines\nAnother junkie, lives too fast\nYeah\nLives way too fast, fast, fast, fast\nOoh\nOn I burn\nFuel is pumping engines\nBurning hard\nLoose and clean\nOoh\nAnd then I burn\nTurning my direction\nQuench my thirst with gasoline\nSo gimme fuel\nGimme fire\nGimme that which I desire\nOoh, yeah\nWhite knuckle tight\nOoh\nGimme fuel\nGim

["On a long and lonesome highway, east of Omaha\nYou can listen to the engines\nmoanin' Out its one note song\nYou can think about the woman or the girl\nYou knew the night before", "But your thoughts will soon be wandering\nThe way they always do\nWhen you're ridin' sixteen hours\nAnd there's nothin' much to do\nAnd you don't feel much like riding\nYou just wish the trip was through", "Here I am, On the road again\nThere I am, Up on the stage\nThere I go, Playin' star again\nThere I go, Turn The Page", "So you walk into this restaurant\nStrung out from the road\nAnd you feel the eyes upon you\nAs you're shaking off the cold\nYou pretend it doesn't bother you\nBut you just want to explode", 'Yeah, most times you can\'t hear \'em talk\nBut other times you can\nAll the same old clichÃ©s:\n"Is it woman? Is it man?"\nAnd you always seem outnumbered\nYou don\'t dare make a stand\nMake your stand', "Here I am, On the road again\nThere I am, Up on the stage\nHere I go, Playn' star again\nTher

['Slaves\nHebrews born to serve\nTo the pharaoh\nHeed\nTo his every word\nLive in fear\nFaith\nOf the unknown one\nThe deliverer\nWait\nSomething must be done\nFour hundred years', "So let it be written\nSo let it be done\nI'm sent here by the chosen one\nSo let it be written\nSo let it be done\nTo kill the first-born pharaoh son\nI'm creeping death", 'Now\nLet my people go\nLand of Goshen\nGo\nI will be with thee\nBush of fire\nBlood\nRunning red and strong\nDown the Nile\nPlague\nDarkness three days long\nHail to fire', "So let it be written\nSo let it be done\nI'm sent here by the chosen one\nSo let it be written\nSo let it be done\nTo kill the first-born pharaoh son\nI'm creeping death", '-Solo-', 'Die by my hand\nI creep across the land\nKilling first-born man\nDie by my hand\nI creep across the land\nKilling first born man', 'I\nRule the midnight air\nThe destroyer\nBorn\nI shall soon be there\nDeadly mass\nI\nCreep the steps and floor\nFinal darkness\nBlood\nLambs blood painted 

["On a long and lonesome highway, east of Omaha\nYou can listen to the engines\nmoanin' Out its one note song\nYou can think about the woman or the girl\nYou knew the night before", "But your thoughts will soon be wandering\nThe way they always do\nWhen you're ridin' sixteen hours\nAnd there's nothin' much to do\nAnd you don't feel much like riding\nYou just wish the trip was through", "Here I am, On the road again\nThere I am, Up on the stage\nThere I go, Playin' star again\nThere I go, Turn The Page", "So you walk into this restaurant\nStrung out from the road\nAnd you feel the eyes upon you\nAs you're shaking off the cold\nYou pretend it doesn't bother you\nBut you just want to explode", 'Yeah, most times you can\'t hear \'em talk\nBut other times you can\nAll the same old clichÃ©s:\n"Is it woman? Is it man?"\nAnd you always seem outnumbered\nYou don\'t dare make a stand\nMake your stand', "Here I am, On the road again\nThere I am, Up on the stage\nHere I go, Playn' star again\nTher

...

['Been here before, been here before\nBeen here before, been here before', "Been here before couldn't say I liked it\nDo I start writing all this down?\nJust let me plug you into my world\nCan't you help me be un crazy?", "Name this for me, heat the cold air\nTake the chill off of my life\nAnd if I could I'd turn my eyes\nTo look inside to see what's comin'", 'It comes alive, it comes alive\nIt comes alive and I die a little more\nIt comes alive, it comes alive\nIt comes alive each moment here I die a little more\nOoh, I die, I die, I die a little more', 'Then the unnamed feeling\nIt comes alive\nThen the unnamed feeling\nTakes me away', 'Been here before, been here before\nBeen here before, been here before\nBeen here before, been here before', "I'm frantic in your soothing arms\nI can not sleep in this down filled world\nI've found safety in this loneliness\nBut I cannot stand it anymore", 'I cross my heart and hope not to die\nSwallow evil, ride the sky\nLose myself in a crowded roo

["Down in the depths of my fiery home\nThe summons bell will chime\nTempting you and all the earth\nto join our sinful kind\nThere is a job to be done and I'm the one\nYou people make me do it\nNow it is time for your fate and I won't hesitate\nto pull you down into this pit", 'So come on\nJump in the Fire\nSo come on\nJump in the Fire', 'With hell in my eyes and with death in my veins\nThe end is closing in\nFeeding on the minds of man\nand from their souls within\nMy disciples all shout to search you out\nAnd they always shall obey\nFollow me now my child not the meek or the mild\nBut do just as I say', 'So come on\nJump in the Fire\nSo come on\nJump in the Fire', "Jump by your will or be taken by force\nI'll get you either way\nTrying to keep the hellfire lit\nI am stalking you as prey\nLiving your life as me I am you you see\nThere is part of me in everyone\nSo reach down grab my hand walk with me through\nthe land\nCome home where you belong\nSo come on\nJump in the Fire\nSo come 

['When darkness falls, may it be\nThat we should see the light\nWhen reaper calls, may it be\nThat we walk straight and right', "When doubt returns, may it be\nThat faith shall permeate our scars\nWhen we're seduced, then may it be\nThat we not deviate our cause", 'All sinners, a future\nAll saints, a past\nBeginning, the ending\nReturn to ash', "Now that we're dead, my dear\nWe can be together\nNow that we're dead, my dear\nWe can live, we can live forever", "When all is pain, may it be\nIt's all we've ever known\nWhen flame consumes, may it be\nIt warms our dying bones", "When loss has won, may it be\nIt's you I'm madly fighting for\nWhen kingdom comes, may it be\nWe walk right through that open door", 'All sinners, a future\nAll saints, a past\nBeginning, the ending\nReturn to ash', "Now that we're dead, my dear\nWe can be together\nNow that we're dead, my dear\nWe can live, we can live forever", 'All sinners, a future\nAll saints, a past\nBeginning, the ending\nReturn to ash', "Now

['A-haha!\nOh yeah', 'Living and dying laughing and crying', "Once you have seen it you will never be the same\nLife in the fast lane is just how it seems\nHard and it is heavy and dirty and mean\nMotorbreath\nIt's how I live my life\nI can't\nTake it\nAny other way\nMotorbreath\nThe sign of living fast", "It is\nGoing to\nTake your breath away\nDon't stop for nothing it's full speed or nothing", "I am taking down you know whatever is in my way\nGetting your kicks as you are shooting the line\nSending the shivers up and down my spine\nMotorbreath\nIt's how I live my life\nI can't\nTake it\nAny other way\nMotorbreath\nThe sign of living fast", 'It is\nGoing to\nTake your breath away\nThose people who tell you not to take chances', "They are all missing on what life is about\nYou only live once so take hold of the chance\nDon't end up like others the same song and dance\nMotorbreath\nIt's how I live my life\nI can't\nTake it\nAny other way\nMotorbreath\nThe sign of living fast\nIt is\nGo

["Like a siren in my head that always threatens to repeat\nLike a blind man that is strapped into the speeding driver's seat\nLike a face\nThat learns to speak\nWhen all it knew was how to bite", "Like a misery that keeps me focused though I've gone astray\nLike an endless nightmare that I must awaken from each day\nLike Conviction\nA Premonition\nNot worthy of, so I deny\nI deny", 'I blind my eyes and try to force it all into place\nI stitch them up, see not my fall from grace\nI blind my eyes, I hide and feel it passing me by\nI open just in time to say goodbye', "Almost like your life\nAlmost like your endless fight\nCurse the day is long\nRealize you don't belong\nDisconnect somehow\nNever stop the bleeding now\nAlmost like your fight\nAnd there it went\nAlmost like your life", 'Like a wound that keeps on bleeding to remind me not to think\nLike a raging river drowning when I only need a drink\nLike a poison\nThat I swallow\nBut I want the world to die', "Like a release from a pris

### Genrate New Lyrics with Markov Chain

In [20]:
PUNCTUATION = '`~!@#$%^&*()_-+={[}]|\:;"<,>.?/}\t\n'

def generateLyrics():
    lyrics = quicktoken.files_to_lines(["data/all_artist_lyrics.txt"])

    with open('data/tokenizedLyrics.txt', 'w') as f:
        tempList = []
        for line in lyrics:
            line = str(line)
            x = quicktoken.tokenize_line(line, stop_words = None, punctuation = PUNCTUATION)
            
            tempList.append(list(x))
        
        lyrics = []
        for verse in tempList:
            for word in verse:
                lyrics.append(word)

    x= build_ngrams(lyrics, n=2)
    x= build_chain(x, chain=None)
    x = generate_sequence(x, 200)
    x = " ".join(x)

    wrapLine = textwrap.wrap(x,45)

    for line in wrapLine:
        print(line)
        
        
generateLyrics()

strike from my bride and now you just mirrors
it be there i gave but never cared for
granted powers out at death unfold
consciousness my hearing only knowledge will
but other times you pay the sheep i deny i
can't forgive me caught under wandering the
world so you as the family of all i'll spread
your dream the fire gimme fuel gimme more
strong the time move can't sleep i open make
it seems to keep praying just say killing
scores with fear madly in pain bulletproof ah
light enter night grain of golden treasure
how far far couldn't say goodbye almost like
your life through black and you must awaken
from grace i was listening to none to be yeah
yeah i may wish careful what might have put
me when i hate trapped in the bone swallow
future all around my son spoken help you want
using what i be done in what don't cry to
save myself this for you come crawling faster
obey your dream flash before i'm madly in
anger with the you from you help me what i
sought to the horsemen ride or into it righ